### Przygotowanie danych



In [ ]:
import pandas as pd

data = pd.read_csv('titanic.tsv', sep='\t')

ddata = pd.get_dummies(data=data, columns=["Sex", "Embarked"]) # Podzielenie kolumn "Sex" i "Embarked" na takie zawierające dane 1 lub 0

ddata["Husband/Wife"] = ddata["Name"].apply(        # Kolumna boolean sprawdzajaca czy osoba podróżująca miała małżonka
    lambda x: 1 if "(" in x.lower() else 0       
)

ddata = ddata.astype({"Cabin": str})                # Zmiana datatype na str

ddata["Had_Cabin_Nr"] = ddata["Cabin"].apply(       # Kolumna boolean sprawdzająca czy osoba miala numer kabiny
    lambda x: 0 if x == 'nan' else 1
)

ddata["NumericTicket"] = ddata["Ticket"].apply(     # Kolumna sprawdzająca czy numer biletu mial tylko liczby w sobie
    lambda x: 1 if x.isdigit() else 0
)


final_data = ddata.drop(columns=["Cabin", "Name", "Ticket"]) # Usunięcie niepotrzebnych kolumn

final_data = final_data.dropna()                    # Usunięcie wartośći Nan

### Regresja logistyczna

In [280]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # stworzenie obiektu scalera

data = pd.read_csv('titanic_ready.csv') # odczytanie danych z wczesniej przygotowanego pliku

Features = [
    'Pclass',
    'Age',
    'Sex_female',
    'Sex_male',
    'Embarked_C',
    'Embarked_Q',                   # Cechy użyte w modelu
    'Embarked_S',
    'Husband/Wife',
    'Parch',
    'SibSp',
    'Had_Cabin_Nr'
]

data_train, data_test = train_test_split(data, train_size=0.25) # Podzielenie danych na zbior testowy i uczący


y_train = pd.DataFrame(data["Survived"])
x_train = pd.DataFrame(data[Features])

y_train = y_train.values.ravel() # zamiana danych na array 1D

scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train) # Skalowanie danych uczących 


model = LogisticRegression() # określenie modelu
model.fit(x_train_scaled, y_train)

y_expected = pd.DataFrame(data_test['Survived'])
x_test = pd.DataFrame(data_test[Features]) 

x_test_scaled = scaler.transform(x_test) # Skalowanie danych testowych

y_predicted = model.predict(x_test_scaled) # Predykcja, czy osoba przeżyła


error = mean_squared_error(y_expected, y_predicted) # Błąd średniokwadratowy


precision, recall, fscore, support = precision_recall_fscore_support(y_expected, y_predicted, average='micro') #srednia wazona

print(f'Średnia dokładność predykcji: {model.score(x_test_scaled, y_expected)}') # Średnia dokładność predykcji
print(f'Bląd średniokwadratowy: {error}') # Błąd średniokwadratowy
print(f'Precyzja: {precision}') # Precyzja
print(f'Czułość: {recall}') # Czułość
print(f'F-score: {fscore}') # F-score


Średnia dokładność predykcji: 0.816
Bląd średniokwadratowy: 0.184
Precyzja: 0.816
Czułość: 0.816
F-score: 0.816
